<a href="https://colab.research.google.com/github/edurso/obj-detect/blob/master/obj_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Object Detection Model Training Notebook

structure of project:

```
training-demo/
├─ annotations/
│  └─ label_map.pbtxt
├─ exported-models/
├─ images/
│  ├─ test/
│  └─ train/
├─ models/
│  └─ ${MODEL}/
│     └─ pipeline.config
├─ pre-trained-models/
│  └─ ${MODEL}/
```

## TensorFlow Object Detection API Installation

mount data from google drive

In [ ]:
workdir = '/content/workspace/MyDrive/tensorflow/power-port-targeting/'
mname = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
from google.colab import drive
import os
!mkdir -p /content/workspace
drive.mount('/content/workspace')

clone tf models repo & utility repo (if not already present)

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

if "utils" in pathlib.Path.cwd().parts:
  while "utils" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('utils').exists():
  !git clone --depth 1 https://github.com/edurso/obj-detect /content/utils

install object_detection package

In [ ]:
%%bash

# update pip
pip install --upgrade pip

# check opencv dependency versions
pip uninstall opencv-python-headless==4.5.5.62 
pip install opencv-python-headless==4.1.2.30

# compile protos and install object_detection
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install . --quiet

test installation

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

check tensorflow version

In [ ]:
import tensorflow as tf
print(tf.__version__)

Data Pre-Processing

create tfrecords

In [ ]:
from time import time
os.chdir(workdir)
start = time()

# Generate TFRecord for Training Data
!python /content/utils/scripts/preprocessing/generate-tfrecord.py \
    -x images/train \
    -l annotations/label_map.pbtxt \
    -o annotations/train.record

# Generate TFRecord for Validation Data
!python /content/utils/scripts/preprocessing/generate-tfrecord.py \
    -x images/test \
    -l annotations/label_map.pbtxt \
    -o annotations/test.record

print('records generated in {}s'.format(time()-start))

## Training

start tensroboard monitoring

In [ ]:
os.chdir(workdir)
%load_ext tensorboard
%tensorboard --logdir=models/{mname}

start training job

In [ ]:
os.chdir(workdir)
!python /content/models/research/object_detection/model_main_tf2.py \
    --include masks \
    --model_dir=models/{mname} \
    --pipeline_config_path=models/{mname}/pipeline.config

## Export Trained Model

export model

In [ ]:
os.chdir(workdir)
!python /tf/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path ./models/{model})/pipeline.config \
    --trained_checkpoint_dir ./models/{model}/ \
    --output_directory ./exported-models/trained_model